In [5]:
import torch 
import torchvision
import torch.nn as nn 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F

import sys
import os
import csv
import numpy as np

import torch
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor

from PIL import Image

import pandas as pd
import linecache
import re

from count_vae import CountVAE 
from datasets import STDataset_triple
from datasets import STDataset_no_neighbhors

from vanilla_vae import VanillaVAE
from cae_dcae import CAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)

In [10]:
preprocess_pip = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

metafile = "../data/meta_train.tsv"
dat1 = STDataset_no_neighbhors(metafile, preprocess_img=preprocess_pip)

In [11]:
#model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_model_29.pth"
#count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_nnet_29.pth"
#model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_no_trip/jd_triplet_model_19.pth"
#count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_no_trip/jd_triplet_nnet_19.pth"

model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet/jd_triplet_model_19.pth"
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet/jd_triplet_nnet_19.pth"

num_classes = 20  #Dimension of the Histology representation we want
n_latent = 20
n_latent_concat = num_classes + n_latent

#Initialize pretrained vision model
model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = 1024
model.classifier = nn.Linear(num_ftrs, num_classes)

model.load_state_dict(torch.load(model_load_path))

nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

Using cache found in /mnt/home/thamamsy/.cache/torch/hub/pytorch_vision_v0.6.0


<All keys matched successfully>

In [12]:
model = model.to(device)
nnet_model = nnet_model.to(device)

dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
nnet_model.eval()
model.eval()

# Get values for GP initialisation:
reconst_loss_train_all = torch.tensor([]).to(device)
kl_divergence_train_all = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        concat_z = model(images).to(device)
        reconst_loss_train, kl_divergence_train = nnet_model(data, library, concat_z)
        

        reconst_loss_train_all = torch.cat((reconst_loss_train_all, reconst_loss_train), 0)
        kl_divergence_train_all = torch.cat((kl_divergence_train_all, kl_divergence_train), 0)
        
        print(reconst_loss_train_all.shape)

torch.Size([512])
torch.Size([1024])
torch.Size([1536])
torch.Size([2048])
torch.Size([2560])
torch.Size([3072])
torch.Size([3584])
torch.Size([4096])
torch.Size([4608])
torch.Size([5120])
torch.Size([5632])
torch.Size([6144])
torch.Size([6656])
torch.Size([7168])
torch.Size([7680])
torch.Size([8192])
torch.Size([8704])


KeyboardInterrupt: 

In [13]:
reconst_loss_train_all_cpu = np.array(reconst_loss_train_all.cpu().detach())
kl_divergence_train_all_cpu = np.array(kl_divergence_train_all.cpu().detach())

#pd.DataFrame(reconst_loss_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_trip_sep_29_recons.csv")
#pd.DataFrame(kl_divergence_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_trip_sep_29_kldiv.csv")

#pd.DataFrame(reconst_loss_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_no_trip_recons.csv")
#pd.DataFrame(kl_divergence_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_no_trip_kldiv.csv")

pd.DataFrame(reconst_loss_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_recons.csv")
pd.DataFrame(kl_divergence_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_densenet_kldiv.csv")

Get loss from Joint CAE models

In [10]:
metafile = "../data/meta_train.tsv"
dat1 = STDataset_no_neighbhors(metafile)
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_cae_sep/jcae_triplet_sep_nnet_39.pth"
load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_cae_sep/jcae_triplet_sep_model_39.pth"

#parameters for cae
denoising = False
latent_dim = 100


########## Pretrained Vision Model loading, count VAE initialization ################## 
num_classes = latent_dim  #Dimension of the Histology representation we want
n_latent = 20
n_latent_concat = num_classes + n_latent

model = CAE(latent_dim=latent_dim)
model.load_state_dict(torch.load(load_model_path))


nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

<All keys matched successfully>

In [11]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
model = model.to(device)
nnet_model = nnet_model.to(device)

nnet_model.eval()
model.eval()

# Get values for GP initialisation:
reconst_loss_train_all = torch.tensor([]).to(device)
kl_divergence_train_all = torch.tensor([]).to(device)


with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        #Sample Z- latent space from vanilla vae
        concat_z = model.sample_z(images).to(device)
        reconst_loss_train, kl_divergence_train = nnet_model(data, library, concat_z)
        
        reconst_loss_train_all = torch.cat((reconst_loss_train_all, reconst_loss_train), 0)
        kl_divergence_train_all = torch.cat((kl_divergence_train_all, kl_divergence_train), 0)
        
        print(reconst_loss_train_all.shape)

torch.Size([512])
torch.Size([1024])
torch.Size([1536])
torch.Size([2048])
torch.Size([2560])
torch.Size([3072])
torch.Size([3584])
torch.Size([4096])
torch.Size([4608])
torch.Size([5120])
torch.Size([5632])
torch.Size([6144])
torch.Size([6656])
torch.Size([7168])
torch.Size([7680])
torch.Size([8192])
torch.Size([8704])
torch.Size([9216])
torch.Size([9728])
torch.Size([10240])
torch.Size([10752])
torch.Size([11264])
torch.Size([11776])
torch.Size([12288])
torch.Size([12800])
torch.Size([13312])
torch.Size([13824])
torch.Size([14336])
torch.Size([14848])
torch.Size([15360])
torch.Size([15872])
torch.Size([16384])
torch.Size([16896])
torch.Size([17408])
torch.Size([17920])
torch.Size([18432])
torch.Size([18944])
torch.Size([19456])
torch.Size([19968])
torch.Size([20480])
torch.Size([20992])
torch.Size([21504])
torch.Size([22016])
torch.Size([22528])
torch.Size([23040])
torch.Size([23552])
torch.Size([24064])
torch.Size([24576])
torch.Size([25088])
torch.Size([25600])
torch.Size([26112])


In [12]:
reconst_loss_train_all_cpu = np.array(reconst_loss_train_all.cpu().detach())
kl_divergence_train_all_cpu = np.array(kl_divergence_train_all.cpu().detach())

pd.DataFrame(reconst_loss_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_cae_sep_recons.csv")
pd.DataFrame(kl_divergence_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_cae_sep_kldiv.csv")

Joint VAE

In [15]:
metafile = "../data/meta_train.tsv"
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_vae/jvae_triplet_nnet_29.pth"
load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_vae/jvae_triplet_model_29.pth"

dat1 = STDataset_no_neighbhors(metafile)

# Generate metadata files for train/test set and instantiate Datasets/DataLoaders
img_size = 256
in_channels = 3
latent_dim = 512

# Instantiate VAE model
model = VanillaVAE(in_channels=in_channels, in_width=img_size, latent_dim=latent_dim)
model.load_state_dict(torch.load(load_model_path))

# Instantiate Count VAE model
num_classes = 512
n_latent = 20
n_latent_concat = num_classes + n_latent

nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

<All keys matched successfully>

In [16]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
model = model.to(device)
nnet_model = nnet_model.to(device)

nnet_model.eval()
model.eval()

# Get values for GP initialisation:
reconst_loss_train_all = torch.tensor([]).to(device)
kl_divergence_train_all = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        #Sample Z- latent space from vanilla vae
        mu_sig_z = model.encode(images)
        concat_z = model.reparameterize(mu_sig_z[0], mu_sig_z[1])
        
        reconst_loss_train, kl_divergence_train = nnet_model(data, library, concat_z)
        reconst_loss_train_all = torch.cat((reconst_loss_train_all, reconst_loss_train), 0)
        kl_divergence_train_all = torch.cat((kl_divergence_train_all, kl_divergence_train), 0)
        
        print(reconst_loss_train_all.shape)

torch.Size([512])
torch.Size([1024])
torch.Size([1536])
torch.Size([2048])
torch.Size([2560])
torch.Size([3072])
torch.Size([3584])
torch.Size([4096])
torch.Size([4608])
torch.Size([5120])
torch.Size([5632])
torch.Size([6144])
torch.Size([6656])
torch.Size([7168])
torch.Size([7680])
torch.Size([8192])
torch.Size([8704])
torch.Size([9216])
torch.Size([9728])
torch.Size([10240])


KeyboardInterrupt: 

In [17]:
reconst_loss_train_all_cpu = np.array(reconst_loss_train_all.cpu().detach())
kl_divergence_train_all_cpu = np.array(kl_divergence_train_all.cpu().detach())

pd.DataFrame(reconst_loss_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_vae_recons.csv")
pd.DataFrame(kl_divergence_train_all_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/losses/joint_vae_kldiv.csv")